<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>Exercise 4: Ensemble Learning </h1></td>
    </tr>
  </tbody>
</table>

# Exercise 4: Ensemble method
https://en.wikipedia.org/wiki/Ensemble_learning
<img src="https://upload.wikimedia.org/wikipedia/commons/d/de/Stacking.png">

## Step 1: Load AIdea AOI dataset

In [ ]:
#load AIdea AOI dataset
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='    y_6pkMwLrg05A4f8S5dRzyu4I5j8Q',dest_path='./content', unzip=True)

## Step 2: Import python libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

## Step 3: Load pretrained models:
* m1: auaoi-InceptionResNetV2
* m2: auaoi-InceptionV3
* m3: auaoi-Xception

In [ ]:
#load AIdea AOI trained CNN models
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='     5v1fkLU3EC-xZCIq6mmvYkKyRUq9',dest_path='./model', unzip=True)

In [ ]:
model_files = ['m1.h5', 'm2.h5', 'm3.h5']
renames = ['a', 'b', 'c']
models = []
for file in model_files:
  model = tf.keras.models.load_model(file)
  models.append(model)


## Step 4: read the training set

In [ ]:
import pandas as pd
df_train = pd.read_csv("       ",dtype=sss)
print(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
train_files = df_train.iloc[:,0].values
train_labels = df_train.iloc[:,1].values
print(train_labels[:20])

In [ ]:
import seaborn as sns
g = sns.countplot(train_labels)

In [ ]:
num_classes=6

## Step 5: Set up an ImageDataGenerator and a Train_generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
#from tensorflow.keras.applications.inception_v3 import preprocess_input
img_gen = ImageDataGenerator(preprocessing_function=pppppppppppppppp)

In [ ]:
train_generator = img_gen.flow_from_dataframe(dataframe=df_train,
            directory="          ",
            x_col="  ",
            y_col="     ",
            subset=None,
            batch_size=8,
            shuffle=False,
            class_mode="categorical",
            color_mode="rgb",
            target_size=(299,299))
print(train_generator.n)

In [ ]:
train_generator.class_indices

In [ ]:
if train_generator.n % train_generator.batch_size ==0:
  step_size_train=train_generator.n//train_generator.batch_size
else:
  step_size_train=train_generator.n//train_generator.batch_size + 1
print(step_size_train)

## Step 6: Evaluation of pretrained models with the training set

In [ ]:
##Evaluation of pretrained models
for model in models:
    train_generator.reset()
    model.evaluate_generator(generator=          , steps=          , verbose=1)

## Step 7: Prediction of pretrained models with the training set

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
y_predictions_All = []
for model in models:
    train_generator.reset()
    y_predictions = model.predict_generator(generator=ttttt_generator, steps=ssssssssss, verbose=1)
    y_predictions_All.append(y_predictions)

## Step 8: Confusion matrices of pretrained models with the training set

In [ ]:
labels = train_labels.astype(int)
print(labels[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
predicts_all = []
for y_predictions in y_predictions_All:
    predicts = np.argmax(y_predictions,axis=1)
    predicts_all.append(predicts)
    confusion=confusion_matrix(labels, predicts)
    print(confusion)

## Step 9: Show differences of predictions between the models

In [ ]:
for i in range(len(labels)):
    label=labels[i]
    pred0=predicts_all[0][i]
    pred1=predicts_all[1][i]
    pred2=predicts_all[2][i]
    if label!=pred0 or label!=pred1 or label!=pred2:
        print(f'{label}->({pred0}, {pred1}, {pred2})')

## Step 10: Ensemble predictions by indiviual models

In [ ]:
y_predictions_ensemble = y_predictions_All[0]+y_predictions_All[1]+y_predictions_All[2]
print(y_predictions_ensemble[:2])
type(y_predictions)

In [ ]:
predicts_ensemble = np.arggggg(y_predictions_ensemble,axis=1)
print(predicts_ensemble[0:10])

## Step 11: Show differences of predictions by the ensembled models

In [ ]:
for i in range(len(labels)):
    label=labels[i]
    pred0=predicts_all[0][i]
    pred1=predicts_all[1][i]
    pred2=predicts_all[2][i]
    predx=predicts_ensemble[i]
    if label!=pred0 or label!=pred1 or label!=pred2:
        print(f'{label}->({pred0}, {pred1}, {pred2})=>{predx}')

In [ ]:
confusion=confusion_matrix(labels, xxxxxxxxxxxxx)
print(confusion)

In [ ]:
overkill= []
underkill = []
for i in range(train_num):
  if labels[i] == 0 and predicts_ensemble[i] !=0:
    overkill.append(i)
  if labels[i] != 0 and predicts_ensemble[i] ==0:
    underkill.append(i)
print('# of overkill= {}; # of underkill= {} '.format(len(overkill), len(underkill)))

## Step 12: Preparing test images

In [ ]:
df_test = pd.read_csv("tttt.csv",dtype=str)
print(df_test.shape)

In [ ]:
df_test.head()

In [ ]:
test_files  = df_test.iloc[:,0].values
test_labels = df_test.iloc[:,1].values
print(test_labels[:10])

In [ ]:
img_gen2 = ImageDataGenerator(horizontal_flip=False, vertical_flip=False, preprocessing_function=)
test_generator = img_gen2.flow_from_dataframe(dataframe=df_test,
            directory="xxxxxxxxxxx",
            x_col="ID",
            y_col="Label",
            batch_size=32,
            shuffle=False,
            class_mode=None,
            target_size=(299,299))

In [ ]:
if test_generator.n % test_generator.batch_size ==0:
  step_size_test=test_generator.n//test_generator.batch_size
else:
  step_size_test=test_generator.n//test_generator.batch_size + 1
print(step_size_test)

## Step 13: Prediction of pretrained models with the testing set

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
y_predictions_All = []
for model in models:
    test_generator.reset()
    y_predictions = model.predict_generator(generator=tttt_generator, steps=step_size_tttt,verbose=1)
    y_predictions_All.append(y_predictions)

## Step 14: Ensemble Predictions

In [ ]:
y_predictions_ensemble = y_predictions_All[0]+y_predictions_All[1]+y_predictions_All[2]
y_predictions_ensemble.shape

In [ ]:
predicts_ensemble=np.max(y_predictions_ensemble,axis=1)

## Step 15: Output the predictions

In [ ]:
df_out = pd.DataFrame(df_test)
df_out.shape

In [ ]:
df_out['Label'] =  
df_out.to_csv("0627-ensemble.csv", index=False)